In [20]:
%matplotlib inline

In [21]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import numpy as np
import csv
import cv2
import os
import tarfile
from urllib.parse import urlparse
from urllib.request import urlretrieve
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
cudnn.benchmark = True
plt.ion()   # interactive mode

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [23]:
%%bash
pwd
unzip "../input/isic-ds/ISIC_2019_Training_Input.zip"

In [24]:
#helper function
def get_label(row):
  for c in labels.columns:
    if row[c]==1:
      return c

labels = pd.read_csv('../input/isic-ds/ISIC_2019_Training_GroundTruth.csv')
labels['l']=labels.apply(get_label, axis=1)
X=labels.drop(['l','UNK'], axis=1)
y = labels['l']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.21, random_state=42,stratify=y)
X_train['image'] = 'ISIC_2019_Training_Input/'+X_train['image']+ '.jpg'
X_test['image'] = 'ISIC_2019_Training_Input/'+X_test['image']+ '.jpg'
X_test.to_csv('validation.csv', index=False)
X_train.to_csv('training.csv', index=False)
train_path = 'training.csv'
class_path = 'validation_class.csv'
eval_path = 'validation.csv'
test_meta = 'ISIC_2019_Test_Metadata.csv'
train_img_path = 'ISIC_2019_Training_Input'

In [25]:
labels

In [26]:
class_names = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']

def read_object_labels_csv(file,phase, header=True):
    img_t=[]
    if phase == 'Train':
      dir = 'training.csv'
    elif phase == "Valid":
      dir = 'validation.csv'
    with open(dir, "r") as f:
        csvReader = csv.reader(f)
        next(csvReader, None)
        k=0
        for line in csvReader:
            k+=1
            image_name= line[0]
            label = line[1:9]
            label = torch.from_numpy(np.array(label, dtype=np.float64))
            item = (image_name,label)
            img_t.append(item)
    return img_t


class isic19(Dataset):
    def __init__(self, root, phase, csv_path,transform=None):
        self.root = os.path.abspath(root)
        self.phase = phase
        self.transform = transform
        self.path_images = self.root
        file_csv = pd.read_csv(csv_path)
        self.classes = class_names
        self.images = read_object_labels_csv(file_csv,phase)
        print('[dataset] classification phase={} number of classes={}  number of images={}'.format(phase, len(self.classes), len(self.images)))

    def __getitem__(self, index):
        filename, target = self.images[index]
        filename = filename
        img = Image.open(filename).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        data = {'image':img, 'target': target}
        return data

    def __len__(self):
        return len(self.images)

    def get_number_classes(self):
        return len(self.classes)


In [27]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
train_dataset = isic19(train_img_path, transform=data_transforms['train'], phase='Train',csv_path=train_path)
val_dataset = isic19(train_img_path, transform=data_transforms['val'],phase='Valid',csv_path=eval_path)
image_datasets ={'train': train_dataset, 'val': val_dataset}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,shuffle=True, num_workers=2)for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [28]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.6678, 0.5298, 0.5245])
    std = np.array([0.1333, 0.1476, 0.1590])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
data = next(iter(dataloaders['train']))
ps = data['target'].numpy()
l = np.where(ps==1.0)
l = l[1].astype(int)
# Make a grid from batch
out = torchvision.utils.make_grid(data['image'])
imshow(out, title=[class_names[x] for x in l])

In [29]:
from PIL import Image
from matplotlib.pyplot import imshow
from torchvision import models, transforms
from torchvision.utils import save_image

In [30]:
model=models.resnet18(pretrained=False)
model.fc=nn.Sequential(nn.Linear(512, 8), nn.Sigmoid())
checkpoint = torch.load('../input/resnet18/RESNET18_wt_epoch_13_batch_312_auc_0.9155.pt',map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'],strict=False)
del checkpoint
model.to(device)

In [31]:
model1=models.densenet121(pretrained=False)
print(model1)
model1.classifier=nn.Sequential(nn.Linear(1024, 8), nn.Sigmoid())
checkpoint = torch.load('../input/isic-densenet121/DENSENET121_wt_epoch_17_batch_1250_auc_0.9298.pt',map_location=torch.device('cpu'))
model1.load_state_dict(checkpoint['model_state_dict'],strict=False)
del checkpoint
model1.to(device)

In [32]:
model2=models.resnet18(pretrained=False)
model2.fc=nn.Sequential(nn.Linear(512, 8), nn.Sigmoid())
checkpoint = torch.load('../input/isicasl/ASLRESNET18_wt_epoch_11_batch_312_auc_0.8514.pt',map_location=torch.device('cpu'))
model2.load_state_dict(checkpoint['model_state_dict'],strict=False)
del checkpoint
model2.to(device)

In [33]:
def validate(model):
    model.eval()
    
    pred_lst=[]
    gt_lst=[]
    for i, sample in enumerate(dataloaders['val']):    
        images=sample['image']
        labels=sample['target']
        del sample
        images = images.to(device)
        #labels = labels.to(device)
        labels=labels.numpy()
        outputs=model(images)
        outputs=outputs.detach()#.cpu().numpy()
        outputs=outputs.cpu()
        outputs=outputs.numpy()
        pred_lst.append(outputs)
        gt_lst.append(labels)
        del labels, outputs, images
             
    pred_lst=np.concatenate(pred_lst, axis=0)
    gt_lst=np.concatenate(gt_lst, axis=0)
        
    auc_lst=[]
    acc_lst=[]
    for k in range(0,8):
        tmp_gt=gt_lst[:, k]
        tmp_pred=pred_lst[:,k]
        tmp_acc=accuracy_score(tmp_gt, tmp_pred.round())
        acc_lst.append(tmp_acc)
        tmp_auc=roc_auc_score(tmp_gt, tmp_pred)
        auc_lst.append(tmp_auc)
            
    auc_lst=np.array(auc_lst)
    acc_lst=np.array(acc_lst)
    print('\n')
    print(auc_lst)
    print('\n')
    print(acc_lst)
    auc=np.mean(auc_lst)
    acc = np.mean(acc_lst)
    print(auc)
    print(acc)
    f1s=0.0
    model.train()
    return auc,acc,f1s

In [34]:
image = Image.open('ISIC_2019_Training_Input/ISIC_0033754.jpg').convert('RGB')
imshow(image)
image = image.resize((224, 224), Image.ANTIALIAS)

print(image.size)
image = np.array(image)
image2 = np.float32(image) / 255

In [35]:
image1 = Image.open('ISIC_2019_Training_Input/ISIC_0033754.jpg').convert('RGB')
preprocess = data_transforms['val']
tensor = preprocess(image1)
tensor2 = tensor.unsqueeze(0)
tensor2 = tensor2.float()

In [39]:
outputs=model1(tensor2.to(device))

Actual Label

In [40]:
outputs

In [41]:
image = Image.open('ISIC_2019_Training_Input/ISIC_0033854.jpg').convert('RGB')
imshow(image)
image = image.resize((224, 224), Image.ANTIALIAS)

print(image.size)
image = np.array(image)
image2 = np.float32(image) / 255

In [45]:
image1 = Image.open('ISIC_2019_Training_Input/ISIC_0068363.jpg').convert('RGB')
preprocess = data_transforms['val']
tensor = preprocess(image1)
tensor2 = tensor.unsqueeze(0)
tensor2 = tensor2.float()

In [46]:
outputs=model1(tensor2.to(device))

Actual Label


In [47]:
outputs